In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-3.1.2-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
import sys, tempfile, urllib.request
from pyspark.sql.functions import *

In [6]:
BASE_DIR = '/tmp'
CORONA_DATA_FILE=os.path.join(BASE_DIR, 'corona_data.csv')

In [7]:
corona_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv',CORONA_DATA_FILE)

In [8]:
!ls /tmp

blockmgr-8068b344-e7c4-4f41-8b5b-f840bfca7221
corona_data.csv
dap_multiplexer.053f15aefd59.root.log.INFO.20210809-134704.51
dap_multiplexer.INFO
debugger_19vvwwaz8x
hsperfdata_root
initgoogle_syslog_dir.0
spark-7d6b201e-5956-408d-9282-6ce9b4e8d802
spark-8c40db0b-1ff7-43ea-8bc4-714e22cce820


In [9]:
corona_df=spark.read.option("inferSchema","true").csv("/tmp/corona_data.csv",header=True)
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [10]:
corona_df.count()

28143

In [11]:
import pyspark.sql.functions as F
corona_max_df=corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),
               on=["Country","State_cleaned","Date"],how="inner")

In [12]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [13]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin("Australia","China")).groupBy("Country").sum().show(100)
#grouping by country, we show the sum of Recovered and sum of Confirmed cases in China and Australia

+---------+--------------+--------------+
|  Country|sum(Confirmed)|sum(Recovered)|
+---------+--------------+--------------+
|    China|         81250|         71266|
|Australia|           791|            26|
+---------+--------------+--------------+



In [18]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin("Australia","China")).cube("Country").sum().sort(desc("Country")).show()
#similar to the above, this only additionally shows the total of confirmed cases and recoved cases across both China & Australia combined, 
#in a descending order, such that the total is displayed at the bottom
#using cube eliminates the need to groupby a column and then perform an aggregation fucntion like sum on the same to achieve the same results

+---------+--------------+--------------+
|  Country|sum(Confirmed)|sum(Recovered)|
+---------+--------------+--------------+
|    China|         81250|         71266|
|Australia|           791|            26|
|     null|         82041|         71292|
+---------+--------------+--------------+



In [22]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin("Australia","Canada")).cube("Country","State_cleaned").sum().sort(desc("Country")).show()
#this shows the sum of confirmed cases and recovered cases across a combination of state and country. 
#The cumulative values are shown at the end, once again across the combination of state and country

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|   Canada|             Alberta|           146|             0|
|   Canada|             Ontario|           308|             5|
|   Canada|       New Brunswick|            11|             0|
|   Canada|      Grand Princess|            10|             0|
|   Canada|Newfoundland and ...|             4|             0|
|   Canada|        Saskatchewan|            20|             0|
|   Canada|              Quebec|           139|             0|
|   Canada|    British Columbia|           271|             4|
|   Canada|Prince Edward Island|             2|             0|
|   Canada|            Manitoba|            17|             0|
|   Canada|                null|           943|             9|
|   Canada|         Nova Scotia|            15|             0|
|Australia|          Queensland|           184|        

In [25]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin("Australia","Canada")).rollup("Country","State_cleaned").sum().sort(asc("Country")).show()
#rollup does a better job for our desired function of displaying a summed up view of recovered and confirmed cases at the country & state level
#and doesn't create a confusing multitude of combinations
#CUBE is for all combinations while ROLLUP is for sub-totals on only the groups

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|     null|                null|          1734|            35|
|Australia|   Western Australia|            64|             0|
|Australia|     New South Wales|           353|             4|
|Australia|            Tasmania|            10|             3|
|Australia|          Queensland|           184|             8|
|Australia|            Victoria|           121|             8|
|Australia|From Diamond Prin...|             0|             0|
|Australia|                null|           791|            26|
|Australia|  Northern Territory|             3|             0|
|Australia|Australian Capita...|             6|             0|
|Australia|     South Australia|            50|             3|
|   Canada|      Grand Princess|            10|             0|
|   Canada|Newfoundland and ...|             4|        

In [14]:
corona_max_df.corr("Confirmed","Recovered")

0.8052934236742176

In [15]:
corona_max_df.cache()

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string]

In [16]:
%timeit corona_max_df.count()
#first iteration in executes the DAG & loads data into memory, next iteration onwards it uses the cached data

The slowest run took 34.29 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 78.1 ms per loop


In [26]:
from pyspark import StorageLevel
corona_max_df.persist(StorageLevel.MEMORY_AND_DISK) #keep the data in memory & disk. This is useful for very large datasets, but for this
#small dataset, only memory option wouldn't have caused any problems
#The below creates only a DAG, execution is performed only in next step, when an ACTION is invoked

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string]

In [27]:
%timeit corona_max_df.count() #this is faster than CACHE

10 loops, best of 5: 53.3 ms per loop


In [28]:
pd=corona_df.toPandas()

In [29]:
pd.corr()

,_c0,Lat,Long,Confirmed,Death,Recovered
_c0,1.000000,-0.001594,-0.005303,0.046789,0.043155,0.042857
Lat,-0.001594,1.000000,-0.383389,0.008531,0.007041,0.000778
Long,-0.005303,-0.383389,1.000000,0.098893,0.074918,0.079047
Confirmed,0.046789,0.008531,0.098893,1.000000,0.963376,0.853924
Death,0.043155,0.007041,0.074918,0.963376,1.000000,0.857292
Recovered,0.042857,0.000778,0.079047,0.853924,0.857292,1.000000


In [30]:
corona_max_df.createOrReplaceTempView("corona")

In [31]:
spark.sql("select * from corona").show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [32]:
spark.sql("select * from corona where Country in ('Australia','Canada') order by Country").show()

+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|  Country|       state_cleaned|      Date|  _c0|               State|               Lat|     Long|Confirmed|Death|Recovered|City|
+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|Australia|     New South Wales|2020-03-20|27672|     New South Wales|          -33.8688| 151.2093|      353|    6|        4|null|
|Australia|            Victoria|2020-03-20|27673|            Victoria|          -37.8136| 144.9631|      121|    0|        8|null|
|Australia|   Western Australia|2020-03-20|27715|   Western Australia|          -31.9505| 115.8605|       64|    1|        0|null|
|Australia|          Queensland|2020-03-20|27674|          Queensland|          -28.0167|    153.4|      184|    0|        8|null|
|Australia|            Tasmania|2020-03-20|27727|            Tasmania|          -41

In [33]:
spark.sql("select * from corona where Country in ('Australia','Canada') sort by Confirmed").show()

+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|  Country|       state_cleaned|      Date|  _c0|               State|               Lat|     Long|Confirmed|Death|Recovered|City|
+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|Australia|From Diamond Prin...|2020-03-20|27688|From Diamond Prin...|           35.4437|  139.638|        0|    0|        0|null|
|   Canada|Prince Edward Island|2020-03-20|28100|Prince Edward Island|           46.5107| -63.4168|        2|    0|        0|null|
|Australia|  Northern Territory|2020-03-20|27737|  Northern Territory|          -12.4634| 130.8456|        3|    0|        0|null|
|   Canada|Newfoundland and ...|2020-03-20|28099|Newfoundland and ...|           53.1355| -57.6604|        4|    0|        0|null|
|Australia|Australian Capita...|2020-03-20|28068|Australian Capita...|          -35

In [36]:
spark.sql("select Country,state_cleaned, SUM(Confirmed), SUM(Recovered) from corona where Country in ('Australia','Canada') GROUP BY ROLLUP (Country, state_cleaned) order by Country DESC").show()

+---------+--------------------+--------------+--------------+
|  Country|       state_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|   Canada|             Alberta|           146|             0|
|   Canada|Newfoundland and ...|             4|             0|
|   Canada|Prince Edward Island|             2|             0|
|   Canada|              Quebec|           139|             0|
|   Canada|       New Brunswick|            11|             0|
|   Canada|        Saskatchewan|            20|             0|
|   Canada|         Nova Scotia|            15|             0|
|   Canada|                null|           943|             9|
|   Canada|             Ontario|           308|             5|
|   Canada|    British Columbia|           271|             4|
|   Canada|            Manitoba|            17|             0|
|   Canada|      Grand Princess|            10|             0|
|Australia|From Diamond Prin...|             0|        

In [35]:
spark.sql("select Country,state_cleaned, SUM(Confirmed), SUM(Recovered) from corona where Country in ('Australia','Canada') GROUP BY CUBE (Country, state_cleaned) order by Country").show()

+-------+--------------------+--------------+--------------+
|Country|       state_cleaned|sum(Confirmed)|sum(Recovered)|
+-------+--------------------+--------------+--------------+
|   null|    British Columbia|           271|             4|
|   null|     New South Wales|           353|             4|
|   null|             Ontario|           308|             5|
|   null|      Grand Princess|            10|             0|
|   null|Newfoundland and ...|             4|             0|
|   null|       New Brunswick|            11|             0|
|   null|            Tasmania|            10|             3|
|   null|        Saskatchewan|            20|             0|
|   null|             Alberta|           146|             0|
|   null|Prince Edward Island|             2|             0|
|   null|            Victoria|           121|             8|
|   null|Australian Capita...|             6|             0|
|   null|            Manitoba|            17|             0|
|   null|          Queen